In [ ]:
import numpy as np
import pandas as pd
import torch
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
import torch.optim as optim

# -------------------------
# Fix Random Seeds
# -------------------------
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

# -------------------------
# Data Prep
# -------------------------
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor  = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32)).view(-1, 1)
y_test_tensor  = torch.from_numpy(y_test.astype(np.float32)).view(-1, 1)

# -------------------------
# Model with Sigmoid
# -------------------------
class MySimpleNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()   # sigmoid is here
        )
    def forward(self, x):
        return self.model(x)

model = MySimpleNN(X_train.shape[1])

# -------------------------
# Loss & Optimizer
# -------------------------
loss_function = nn.BCELoss()   # requires sigmoid output
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -------------------------
# Training
# -------------------------
epochs = 200
for epoch in range(epochs):
    y_pred = model(X_train_tensor)             # already probabilities
    loss = loss_function(y_pred, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# -------------------------
# Evaluation
# -------------------------
with torch.no_grad():
    y_pred = model(X_test_tensor)             # already probabilities
    y_pred_cls = (y_pred > 0.5).float()
    acc = (y_pred_cls == y_test_tensor).float().mean()
    print(f"Accuracy: {acc.item()*100:.2f}%")

Epoch [50/200], Loss: 0.1558
Epoch [100/200], Loss: 0.0587
Epoch [150/200], Loss: 0.0381
Epoch [200/200], Loss: 0.0252
Accuracy: 98.25%


In [ ]:
new_data = X_test[0:1]

new_tensor = torch.tensor(new_data, dtype=torch.float32).view(1, -1)

model.eval()
with torch.no_grad():
    y_proba = model(new_tensor)
    y_class = (y_proba > 0.5).float()
    print("Predicted Probability:", y_proba.item())
    print("Predicted Class:", int(y_class.item()))


Predicted Probability: 0.05342916399240494
Predicted Class: 0
